In [15]:
import os
import json
import math
import numpy as np
import copy
from scipy.spatial.transform import Rotation

In [16]:
SAVE_PATH = os.path.join(os.getcwd(), '../data/jackal_training_data')
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)
if not os.path.exists(os.path.join(SAVE_PATH, 'images')):
    os.makedirs(os.path.join(SAVE_PATH, 'images'))

In [17]:
transforms = {}
transforms["focal_length"] = 4.74 #Focal Length in mm
transforms["fov_x"] = 1.1519 * 0.5 #FOV in radians
transforms["w"] = 1280  # image width
transforms["h"] = 720 # image height


transforms["pixel_width"] = 2*math.tan(transforms["fov_x"]) * transforms["focal_length"] / transforms["w"] #Sensor width in mm
transforms["pixel_height"] = transforms["pixel_width"] #Sensor height in mm
transforms["fl_x"] = transforms["focal_length"]/transforms["pixel_width"] # focal length/ pixel_width
transforms["fl_y"] = transforms["focal_length"]/transforms["pixel_height"]# focal length/ pixel_height
transforms["cx"] = transforms["w"]//2 # optical center in x
transforms["cy"] = transforms["h"]//2 # optical center in y
transforms["camera_model"] = "OPENCV" # camera model

ground_truth = {}
ground_truth = transforms.copy()

In [18]:
print(transforms["pixel_width"])

0.00480958387256315


In [19]:
frames = []
ground_truth_frames = []

starting_pts = [[0.0,-2.5], [0.0, -2], [0, -1.5], [0,-1]]
z_store = [2]
pitch = 0.785
spin = 36

cnt = 0

sd_translation = 0
sd_rotation = 0

for x, y in starting_pts:
    for z in z_store:
        start_transform_matrix = np.eye(4)
        start_transform_matrix[:3, :3] = Rotation.from_rotvec(np.array([pitch, 0, 0])).as_matrix()
        start_transform_matrix[0, 3] = x
        start_transform_matrix[1, 3] = y
        start_transform_matrix[2, 3] = z

        for i in range(spin):
            angle = i * 2 * np.pi / spin
            rot = Rotation.from_rotvec(np.array([0, 0, 1]) * angle).as_matrix()
            rot_matrix = np.eye(4)
            rot_matrix[:3, :3] = rot
            transform_matrix = np.matmul(rot_matrix, start_transform_matrix)

            frame = {}
            file_path = "images/{}.png".format(cnt)

            frame["file_path"] = file_path
            frame["transform_matrix"] = transform_matrix.tolist().copy()
            frames.append(frame.copy())

            
            noise_matrix = np.eye(4)
            noise_matrix[0, 3] = np.random.normal(0, sd_translation)
            noise_matrix[1, 3] = np.random.normal(0, sd_translation)
            noise_matrix[:3, :3] = Rotation.from_rotvec(np.array([0, 0, 1]) * np.random.normal(0, sd_rotation), degrees=True).as_matrix()
            print(transform_matrix)
            transform_matrix = np.matmul(noise_matrix, transform_matrix)
            print(transform_matrix)
            
            frame["transform_matrix"] = transform_matrix.tolist().copy()
            
            ground_truth_frames.append(frame.copy())
            
            
            
            cnt += 1


transforms["frames"] = frames
ground_truth["frames"] = ground_truth_frames

[[ 1.          0.          0.          0.        ]
 [ 0.          0.70738827 -0.70682518 -2.5       ]
 [ 0.          0.70682518  0.70738827  2.        ]
 [ 0.          0.          0.          1.        ]]
[[ 1.          0.          0.          0.        ]
 [ 0.          0.70738827 -0.70682518 -2.5       ]
 [ 0.          0.70682518  0.70738827  2.        ]
 [ 0.          0.          0.          1.        ]]
[[ 0.98480775 -0.12283668  0.1227389   0.43412044]
 [ 0.17364818  0.69664145 -0.69608692 -2.46201938]
 [ 0.          0.70682518  0.70738827  2.        ]
 [ 0.          0.          0.          1.        ]]
[[ 0.98480775 -0.12283668  0.1227389   0.43412044]
 [ 0.17364818  0.69664145 -0.69608692 -2.46201938]
 [ 0.          0.70682518  0.70738827  2.        ]
 [ 0.          0.          0.          1.        ]]
[[ 0.93969262 -0.24194104  0.24174845  0.85505036]
 [ 0.34202014  0.66472754 -0.66419841 -2.34923155]
 [ 0.          0.70682518  0.70738827  2.        ]
 [ 0.          0.          

In [20]:
print(transforms["frames"])
print(ground_truth["frames"])

[{'file_path': 'images/0.png', 'transform_matrix': [[0.9999999999999999, 0.0, 0.0, 0.0], [0.0, 0.7073882691671997, -0.706825181105366, -2.5], [0.0, 0.706825181105366, 0.7073882691671997, 2.0], [0.0, 0.0, 0.0, 1.0]]}, {'file_path': 'images/1.png', 'transform_matrix': [[0.984807753012208, -0.12283668384384822, 0.12273890462804479, 0.4341204441673258], [0.1736481776669303, 0.696641451865745, -0.6960869183768226, -2.4620193825305203], [0.0, 0.706825181105366, 0.7073882691671997, 2.0], [0.0, 0.0, 0.0, 1.0]]}, {'file_path': 'images/2.png', 'transform_matrix': [[0.9396926207859082, -0.2419410372074623, 0.24174844974784898, 0.8550503583141716], [0.3420201433256686, 0.6647275365669334, -0.6641984068703757, -2.349231551964771], [0.0, 0.7068251811053659, 0.7073882691671995, 1.9999999999999998], [0.0, 0.0, 0.0, 1.0]]}, {'file_path': 'images/3.png', 'transform_matrix': [[0.8660254037844386, -0.3536941345835998, 0.3534125905526829, 1.2499999999999998], [0.4999999999999999, 0.6126162114378993, -0.612

In [21]:
#Save transforms
with open(os.path.join(SAVE_PATH,'transforms.json'), 'w') as outfile:
    json.dump(transforms, outfile, indent=4)
with open(os.path.join(SAVE_PATH,'ground_truth.json'), 'w') as outfile:
    json.dump(ground_truth, outfile, indent=4)